In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MultiLabelBinarizer
import torch
import torch.nn as nn
import math
from datetime import datetime

In [17]:
dir_1 = 'datasets/'

In [18]:
df_1a = pd.read_csv(dir_1 + 'movie_meta_data.csv')
df_2a = pd.read_csv(dir_1 + 'imdb-movies.csv')

In [19]:
print(df_1a.shape)
print(df_2a.shape)

(2858, 25)
(10866, 21)


In [20]:
df_3a = pd.merge(df_1a, df_2a, left_on='title', right_on='original_title')
df_3a.shape

(1968, 46)

In [21]:
df_3a.columns

Index(['imdbid', 'title', 'akas', 'year', 'metascore', 'imdb user rating',
       'number of imdb user votes', 'awards', 'opening weekend', 'producers',
       'budget_x', 'script department', 'production companies', 'writers',
       'directors', 'casting directors', 'cast_x', 'countries', 'age restrict',
       'plot', 'plot outline', 'keywords_x', 'genres_x', 'taglines',
       'synopsis', 'id', 'imdb_id', 'popularity', 'budget_y', 'revenue',
       'original_title', 'cast_y', 'homepage', 'director', 'tagline',
       'keywords_y', 'overview', 'runtime', 'genres_y', 'production_companies',
       'release_date', 'vote_count', 'vote_average', 'release_year',
       'budget_adj', 'revenue_adj'],
      dtype='object')

In [22]:
list_col = [    
    'budget_y',
    'writers', 'directors', 'producers', 'cast_x',
    'genres_x',    
    'metascore', 'imdb user rating',        
    'release_date',
    'revenue',
]

In [23]:
df_3b = df_3a[list_col]

In [24]:
df_3c = df_3b.drop(df_3b[df_3b['budget_y'] < 0].index)
df_3d = df_3c.drop(df_3c[df_3c['revenue'] < 0].index)
df_3e = df_3d.drop(df_3d[(df_3d['metascore'] < 0) | (df_3d['metascore'] > 100)].index)
df_3f = df_3e.drop(df_3e[(df_3e['imdb user rating'] < 1) | (df_3e['imdb user rating'] > 10)].index)

In [25]:
df_3b.shape, df_3c.shape, df_3d.shape, df_3e.shape, df_3f.shape

((1968, 10), (1968, 10), (1968, 10), (1764, 10), (1764, 10))

In [26]:
# df_3f['writers'] = df_3f['writers'].apply(lambda x: str(x).split(','))
# df_3f['writers'] = df_3f['writers'].apply(lambda x: [writer.strip() for writer in x])
# col_writers = list(df_3f['writers'])
# d_writer_vs_index = {}
# iw=0
# for writers in col_writers:
#     for w in writers:
#         if w not in d_writer_vs_index.keys():
#             d_writer_vs_index[w] = iw
#             iw+=1
# df_3f['writers'] = df_3f['writers'].apply(lambda x: [d_writer_vs_index[writer] for writer in x])
# w_vocab_size = len(d_writer_vs_index.keys()) + 1
# w_embedding_dim = math.floor(math.log2(w_vocab_size) + 1)
# w_embedding_layer = nn.Embedding(w_vocab_size, w_embedding_dim) 
# df_3f['writers'] = df_3f['writers'].apply(lambda x: [w_embedding_layer(torch.tensor(iw)).detach().numpy() for iw in x])
# df_3f['writers'] = df_3f['writers'].apply(lambda x: np.mean(np.array(x), axis=0))
############################################################################################################
df_3f['writers'] = df_3f['writers'].apply(lambda x: str(x).split(','))
df_3f['writers'] = df_3f['writers'].apply(lambda x: [elem.strip() for elem in x])
df_3f['writers'] = df_3f['writers'].apply(lambda x: x[0:10])
encoder = MultiLabelBinarizer()
one_hot = encoder.fit_transform(df_3f['writers'])



X = torch.Tensor(one_hot)
input_dim=X.shape[1]
output_dim = math.floor(math.log2(input_dim) + 1)
inter_dim = int(0.5*((0.5*input_dim) + (2*output_dim)))
print(X.shape, input_dim, output_dim, inter_dim)

# define the autoencoder architecture
class Autoencoder(nn.Module):
    def __init__(self, dim_in, dim_out, dim_inter):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(dim_in, dim_inter),
            nn.ReLU(),
            nn.Linear(dim_inter, dim_out)
        )
        self.decoder = nn.Sequential(
            nn.Linear(dim_out, dim_inter),
            nn.ReLU(),
            nn.Linear(dim_inter, dim_in)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# create the autoencoder object
autoencoder = Autoencoder(dim_in=input_dim, dim_out=output_dim, dim_inter=inter_dim)

# set loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters())

# train the autoencoder
for epoch in range(101):
    # forward pass
    output = autoencoder(X)
    loss = criterion(output, X)

    # backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, 100, loss.item()))

# get the encoded data
X_encoded = autoencoder.encoder(X).detach().numpy()

newcol = ['w' + str(i) for i in range(output_dim)]
newdf = pd.DataFrame(X_encoded, columns=newcol)

df_3f = pd.concat([df_3f.reset_index(), newdf.reset_index()], axis=1)
print(df_3f.shape)
df_3f.head(3)

torch.Size([1764, 2371]) 2371 12 604
epoch [1/100], loss:0.0063
epoch [101/100], loss:0.0010
(1764, 24)


,index,budget_y,writers,directors,producers,cast_x,genres_x,metascore,imdb user rating,release_date,...,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11
0,0,17000000,"[Steve Koren, Will Ferrell, Chris Kattan]",John Fortenberry,"Marie Cantin, Erin Fraser, Amy Heckerling, Ste...","Will Ferrell, Chris Kattan, Raquel Gardner, Vi...","Comedy, Music, Romance",26,6,10/1/98,...,-0.058453,0.122996,-0.016373,-0.036342,0.008340,-0.030410,-0.026067,-0.042747,-0.074768,-0.066924
1,1,0,"[Oliver Sacks, Steve Levitt]",Irwin Winkler,"Rob Cowan, Roger Paradiso, Irwin Winkler","Val Kilmer, Mira Sorvino, Kelly McGillis, Stev...","Drama, Romance",40,6,1/15/99,...,-0.032193,0.116719,-0.036274,-0.029260,0.019342,-0.032677,-0.023810,-0.065589,-0.097106,-0.054858
2,2,220000000,"[Sydney Newman, Don MacPherson]",Jeremiah S. Chechik,"Susan Ekins, Jerry Weintraub","Ralph Fiennes, Uma Thurman, Sean Connery, Patr...","Action, Adventure, Sci-Fi, Thriller",12,3,4/25/12,...,-0.027255,0.116650,-0.039533,-0.033460,0.019646,-0.023754,-0.015934,-0.033105,-0.084040,-0.066557


In [27]:
# df_3f['directors'] = df_3f['directors'].apply(lambda x: str(x).split(','))
# df_3f['directors'] = df_3f['directors'].apply(lambda x: [director.strip() for director in x])
# col_directors = list(df_3f['directors'])
# d_director_vs_index = {}
# idirector=0
# for directors in col_directors:
#     for d in directors:
#         if d not in d_director_vs_index.keys():
#             d_director_vs_index[d] = idirector
#             idirector+=1
# df_3f['directors'] = df_3f['directors'].apply(lambda x: [d_director_vs_index[director] for director in x])
# d_vocab_size = len(d_director_vs_index.keys()) + 1
# d_embedding_dim = math.floor(math.log2(d_vocab_size) + 1)
# d_embedding_layer = nn.Embedding(d_vocab_size, d_embedding_dim) 
# df_3f['directors'] = df_3f['directors'].apply(lambda x: [d_embedding_layer(torch.tensor(idirector)).detach().numpy() for idirector in x])
# df_3f['directors'] = df_3f['directors'].apply(lambda x: np.mean(np.array(x), axis=0))
############################################################################################################
df_3f['directors'] = df_3f['directors'].apply(lambda x: str(x).split(','))
df_3f['directors'] = df_3f['directors'].apply(lambda x: [elem.strip() for elem in x])
df_3f['directors'] = df_3f['directors'].apply(lambda x: x[0:10])
encoder = MultiLabelBinarizer()
one_hot = encoder.fit_transform(df_3f['directors'])

X = torch.Tensor(one_hot)
input_dim=X.shape[1]
output_dim = math.floor(math.log2(input_dim) + 1)
inter_dim = int(0.5*((0.5*input_dim) + (2*output_dim)))
print(X.shape, input_dim, output_dim, inter_dim)

# define the autoencoder architecture
class Autoencoder(nn.Module):
    def __init__(self, dim_in, dim_out, dim_inter):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(dim_in, dim_inter),
            nn.ReLU(),
            nn.Linear(dim_inter, dim_out)
        )
        self.decoder = nn.Sequential(
            nn.Linear(dim_out, dim_inter),
            nn.ReLU(),
            nn.Linear(dim_inter, dim_in)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# create the autoencoder object
autoencoder = Autoencoder(dim_in=input_dim, dim_out=output_dim, dim_inter=inter_dim)

# set loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters())

# train the autoencoder
for epoch in range(101):
    # forward pass
    output = autoencoder(X)
    loss = criterion(output, X)

    # backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, 100, loss.item()))

# get the encoded data
X_encoded = autoencoder.encoder(X).detach().numpy()

newcol = ['d' + str(i) for i in range(output_dim)]
newdf = pd.DataFrame(X_encoded, columns=newcol)
df_3f = pd.concat([df_3f, newdf], axis=1)
print(df_3f.shape)
df_3f.head(3)

torch.Size([1764, 880]) 880 10 230
epoch [1/100], loss:0.0088
epoch [101/100], loss:0.0012
(1764, 34)


,index,budget_y,writers,directors,producers,cast_x,genres_x,metascore,imdb user rating,release_date,...,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9
0,0,17000000,"[Steve Koren, Will Ferrell, Chris Kattan]",[John Fortenberry],"Marie Cantin, Erin Fraser, Amy Heckerling, Ste...","Will Ferrell, Chris Kattan, Raquel Gardner, Vi...","Comedy, Music, Romance",26,6,10/1/98,...,0.119722,-0.053771,-0.090771,-0.023893,-0.021543,-0.117464,-0.079050,0.058924,-0.016714,0.067209
1,1,0,"[Oliver Sacks, Steve Levitt]",[Irwin Winkler],"Rob Cowan, Roger Paradiso, Irwin Winkler","Val Kilmer, Mira Sorvino, Kelly McGillis, Stev...","Drama, Romance",40,6,1/15/99,...,0.076636,-0.074008,-0.073870,-0.028360,0.006585,-0.103703,-0.098796,0.055264,-0.051437,0.043243
2,2,220000000,"[Sydney Newman, Don MacPherson]",[Jeremiah S. Chechik],"Susan Ekins, Jerry Weintraub","Ralph Fiennes, Uma Thurman, Sean Connery, Patr...","Action, Adventure, Sci-Fi, Thriller",12,3,4/25/12,...,0.073194,-0.076075,-0.062231,0.016069,-0.028275,-0.110439,-0.094791,0.044910,-0.040385,0.061729


In [28]:
# df_3f['producers'] = df_3f['producers'].apply(lambda x: str(x).split(','))
# df_3f['producers'] = df_3f['producers'].apply(lambda x: [producer.strip() for producer in x])
# col_p = list(df_3f['producers'])
# d_p_vs_index = {}
# ip=0
# for ps in col_p:
#     for p in ps:
#         if p not in d_p_vs_index.keys():
#             d_p_vs_index[p] = ip
#             ip+=1
# df_3f['producers'] = df_3f['producers'].apply(lambda x: [d_p_vs_index[p] for p in x])
# p_vocab_size = len(d_p_vs_index.keys()) + 1
# p_embedding_dim = math.floor(math.log2(p_vocab_size) + 1)
# p_embedding_layer = nn.Embedding(p_vocab_size, p_embedding_dim) 
# df_3f['producers'] = df_3f['producers'].apply(lambda x: [p_embedding_layer(torch.tensor(ip)).detach().numpy() for ip in x])
# df_3f['producers'] = df_3f['producers'].apply(lambda x: np.mean(np.array(x), axis=0))
############################################################################################################
df_3f['producers'] = df_3f['producers'].apply(lambda x: str(x).split(','))
df_3f['producers'] = df_3f['producers'].apply(lambda x: [elem.strip() for elem in x])
df_3f['producers'] = df_3f['producers'].apply(lambda x: x[0:10])
encoder = MultiLabelBinarizer()
one_hot = encoder.fit_transform(df_3f['producers'])

X = torch.Tensor(one_hot)
input_dim=X.shape[1]
output_dim = math.floor(math.log2(input_dim) + 1)
inter_dim = int(0.5*((0.5*input_dim) + (2*output_dim)))
print(X.shape, input_dim, output_dim, inter_dim)

# define the autoencoder architecture
class Autoencoder(nn.Module):
    def __init__(self, dim_in, dim_out, dim_inter):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(dim_in, dim_inter),
            nn.ReLU(),
            nn.Linear(dim_inter, dim_out)
        )
        self.decoder = nn.Sequential(
            nn.Linear(dim_out, dim_inter),
            nn.ReLU(),
            nn.Linear(dim_inter, dim_in)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# create the autoencoder object
autoencoder = Autoencoder(dim_in=input_dim, dim_out=output_dim, dim_inter=inter_dim)

# set loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters())

# train the autoencoder
for epoch in range(101):
    # forward pass
    output = autoencoder(X)
    loss = criterion(output, X)

    # backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, 100, loss.item()))

# get the encoded data
X_encoded = autoencoder.encoder(X).detach().numpy()

newcol = ['p' + str(i) for i in range(output_dim)]
newdf = pd.DataFrame(X_encoded, columns=newcol)
df_3f = pd.concat([df_3f, newdf], axis=1)
print(df_3f.shape)
df_3f.head(3)

torch.Size([1764, 5388]) 5388 13 1360
epoch [1/100], loss:0.0058
epoch [101/100], loss:0.0012
(1764, 47)


,index,budget_y,writers,directors,producers,cast_x,genres_x,metascore,imdb user rating,release_date,...,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12
0,0,17000000,"[Steve Koren, Will Ferrell, Chris Kattan]",[John Fortenberry],"[Marie Cantin, Erin Fraser, Amy Heckerling, St...","Will Ferrell, Chris Kattan, Raquel Gardner, Vi...","Comedy, Music, Romance",26,6,10/1/98,...,-0.044625,-0.017956,0.006390,-0.009068,0.053640,0.034079,0.020217,-0.034410,0.019518,0.070436
1,1,0,"[Oliver Sacks, Steve Levitt]",[Irwin Winkler],"[Rob Cowan, Roger Paradiso, Irwin Winkler]","Val Kilmer, Mira Sorvino, Kelly McGillis, Stev...","Drama, Romance",40,6,1/15/99,...,-0.044424,-0.005684,-0.006312,0.006900,0.054985,0.026896,0.012876,-0.036281,0.017652,0.058381
2,2,220000000,"[Sydney Newman, Don MacPherson]",[Jeremiah S. Chechik],"[Susan Ekins, Jerry Weintraub]","Ralph Fiennes, Uma Thurman, Sean Connery, Patr...","Action, Adventure, Sci-Fi, Thriller",12,3,4/25/12,...,-0.038415,-0.018299,0.005070,0.000485,0.053834,0.026724,0.008589,-0.040991,0.022421,0.073969


In [29]:
# df_3f['cast_x'] = df_3f['cast_x'].apply(lambda x: str(x).split(','))
# df_3f['cast_x'] = df_3f['cast_x'].apply(lambda x: [cast.strip() for cast in x])
# col_c = list(df_3f['cast_x'])
# d_c_vs_index = {}
# ic=0
# for cs in col_c:
#     for c in cs:
#         if c not in d_c_vs_index.keys():
#             d_c_vs_index[c] = ic
#             ic+=1
# df_3f['cast_x'] = df_3f['cast_x'].apply(lambda x: [d_c_vs_index[c] for c in x])
# c_vocab_size = len(d_c_vs_index.keys()) + 1
# c_embedding_dim = math.floor(math.log2(c_vocab_size) + 1)
# c_embedding_layer = nn.Embedding(c_vocab_size, c_embedding_dim) 
# df_3f['cast_x'] = df_3f['cast_x'].apply(lambda x: [c_embedding_layer(torch.tensor(ic)).detach().numpy() for ic in x])
# df_3f['cast_x'] = df_3f['cast_x'].apply(lambda x: np.mean(np.array(x), axis=0))
############################################################################################################
df_3f['cast_x'] = df_3f['cast_x'].apply(lambda x: str(x).split(','))
df_3f['cast_x'] = df_3f['cast_x'].apply(lambda x: [elem.strip() for elem in x])
df_3f['cast_x'] = df_3f['cast_x'].apply(lambda x: x[0:10])

encoder = MultiLabelBinarizer()
one_hot = encoder.fit_transform(df_3f['cast_x'])

X = torch.Tensor(one_hot)
input_dim=X.shape[1]
output_dim = math.floor(math.log2(input_dim) + 1)
inter_dim = int(0.5*((0.5*input_dim) + (2*output_dim)))
print(X.shape, input_dim, output_dim, inter_dim)

# define the autoencoder architecture
class Autoencoder(nn.Module):
    def __init__(self, dim_in, dim_out, dim_inter):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(dim_in, dim_inter),
            nn.ReLU(),
            nn.Linear(dim_inter, dim_out)
        )
        self.decoder = nn.Sequential(
            nn.Linear(dim_out, dim_inter),
            nn.ReLU(),
            nn.Linear(dim_inter, dim_in)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# create the autoencoder object
autoencoder = Autoencoder(dim_in=input_dim, dim_out=output_dim, dim_inter=inter_dim)

# set loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters())

# train the autoencoder
for epoch in range(101):
    # forward pass
    output = autoencoder(X)
    loss = criterion(output, X)

    # backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, 100, loss.item()))

# get the encoded data
X_encoded = autoencoder.encoder(X).detach().numpy()

newcol = ['c' + str(i) for i in range(output_dim)]
newdf = pd.DataFrame(X_encoded, columns=newcol)
df_3f = pd.concat([df_3f, newdf], axis=1)
print(df_3f.shape)
df_3f.head(3)

torch.Size([1764, 9062]) 9062 14 2279
epoch [1/100], loss:0.0053
epoch [101/100], loss:0.0011
(1764, 61)


,index,budget_y,writers,directors,producers,cast_x,genres_x,metascore,imdb user rating,release_date,...,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13
0,0,17000000,"[Steve Koren, Will Ferrell, Chris Kattan]",[John Fortenberry],"[Marie Cantin, Erin Fraser, Amy Heckerling, St...","[Will Ferrell, Chris Kattan, Raquel Gardner, V...","Comedy, Music, Romance",26,6,10/1/98,...,-0.020697,-0.025011,-0.006198,0.011103,-0.016822,0.032601,-0.036822,0.022509,0.020666,0.021917
1,1,0,"[Oliver Sacks, Steve Levitt]",[Irwin Winkler],"[Rob Cowan, Roger Paradiso, Irwin Winkler]","[Val Kilmer, Mira Sorvino, Kelly McGillis, Ste...","Drama, Romance",40,6,1/15/99,...,-0.015057,-0.021132,0.003687,-0.004116,-0.024244,0.022939,-0.031910,0.024431,0.023022,0.030792
2,2,220000000,"[Sydney Newman, Don MacPherson]",[Jeremiah S. Chechik],"[Susan Ekins, Jerry Weintraub]","[Ralph Fiennes, Uma Thurman, Sean Connery, Pat...","Action, Adventure, Sci-Fi, Thriller",12,3,4/25/12,...,-0.017451,-0.018689,0.003623,0.003854,-0.022389,0.025489,-0.036089,0.004298,0.021361,0.036925


In [30]:
############################################################################################################
df_3f['genres_x'] = df_3f['genres_x'].apply(lambda x: str(x).split(','))
df_3f['genres_x'] = df_3f['genres_x'].apply(lambda x: [elem.strip() for elem in x])
df_3f['genres_x'] = df_3f['genres_x'].apply(lambda x: x[0:10])
encoder = MultiLabelBinarizer()
one_hot = encoder.fit_transform(df_3f['genres_x'])

In [34]:
pd.DataFrame(one_hot).head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [15]:
X = torch.Tensor(one_hot)
input_dim=X.shape[1]
output_dim = math.floor(math.log2(input_dim) + 1)
inter_dim = int(0.5*((0.5*input_dim) + (2*output_dim)))
print(X.shape, input_dim, output_dim, inter_dim)

# define the autoencoder architecture
class Autoencoder(nn.Module):
    def __init__(self, dim_in, dim_out, dim_inter):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(dim_in, dim_inter),
            nn.ReLU(),
            nn.Linear(dim_inter, dim_out)
        )
        self.decoder = nn.Sequential(
            nn.Linear(dim_out, dim_inter),
            nn.ReLU(),
            nn.Linear(dim_inter, dim_in)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# create the autoencoder object
autoencoder = Autoencoder(dim_in=input_dim, dim_out=output_dim, dim_inter=inter_dim)

# set loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters())

# train the autoencoder
for epoch in range(101):
    # forward pass
    output = autoencoder(X)
    loss = criterion(output, X)

    # backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, 100, loss.item()))

# get the encoded data
X_encoded = autoencoder.encoder(X).detach().numpy()

newcol = ['g' + str(i) for i in range(output_dim)]
newdf = pd.DataFrame(X_encoded, columns=newcol)
df_3f = pd.concat([df_3f, newdf], axis=1)
print(df_3f.shape)
df_3f.head(3)

torch.Size([1764, 22]) 22 5 10
epoch [1/100], loss:0.1903
epoch [101/100], loss:0.1113
(1764, 66)


,index,budget_y,writers,directors,producers,cast_x,genres_x,metascore,imdb user rating,release_date,...,c9,c10,c11,c12,c13,g0,g1,g2,g3,g4
0,0,17000000,"[Steve Koren, Will Ferrell, Chris Kattan]",[John Fortenberry],"[Marie Cantin, Erin Fraser, Amy Heckerling, St...","[Will Ferrell, Chris Kattan, Raquel Gardner, V...","[Comedy, Music, Romance]",26,6,10/1/98,...,0.031148,-0.003272,-0.016168,-0.017277,0.026549,-0.490822,-0.090693,-0.207875,-0.176953,0.181547
1,1,0,"[Oliver Sacks, Steve Levitt]",[Irwin Winkler],"[Rob Cowan, Roger Paradiso, Irwin Winkler]","[Val Kilmer, Mira Sorvino, Kelly McGillis, Ste...","[Drama, Romance]",40,6,1/15/99,...,0.046184,0.006112,-0.003462,-0.011548,0.025018,-0.411669,0.173315,-0.066780,-0.122879,0.591124
2,2,220000000,"[Sydney Newman, Don MacPherson]",[Jeremiah S. Chechik],"[Susan Ekins, Jerry Weintraub]","[Ralph Fiennes, Uma Thurman, Sean Connery, Pat...","[Action, Adventure, Sci-Fi, Thriller]",12,3,4/25/12,...,0.038517,0.004903,-0.006880,-0.009138,0.021588,-0.477744,0.401622,-0.513233,-0.510372,0.576269


In [16]:
df_3f['week_of_year'] = df_3f['release_date'].apply(lambda x: datetime.strptime(x, '%m/%d/%y').isocalendar()[1])
df_3f['budget'] = df_3f['budget_y']
df_3f['imdb_user_rating'] = df_3f['imdb user rating']

In [17]:
df_3f.head(2)

,index,budget_y,writers,directors,producers,cast_x,genres_x,metascore,imdb user rating,release_date,...,c12,c13,g0,g1,g2,g3,g4,week_of_year,budget,imdb_user_rating
0,0,17000000,"[Steve Koren, Will Ferrell, Chris Kattan]",[John Fortenberry],"[Marie Cantin, Erin Fraser, Amy Heckerling, St...","[Will Ferrell, Chris Kattan, Raquel Gardner, V...","[Comedy, Music, Romance]",26,6,10/1/98,...,-0.017277,0.026549,-0.490822,-0.090693,-0.207875,-0.176953,0.181547,40,17000000,6
1,1,0,"[Oliver Sacks, Steve Levitt]",[Irwin Winkler],"[Rob Cowan, Roger Paradiso, Irwin Winkler]","[Val Kilmer, Mira Sorvino, Kelly McGillis, Ste...","[Drama, Romance]",40,6,1/15/99,...,-0.011548,0.025018,-0.411669,0.173315,-0.066780,-0.122879,0.591124,2,0,6


In [18]:
df_4a = df_3f[[
    'budget', 
    'w0', 'w1','w2', 'w3','w4', 'w5','w6', 'w7','w8', 'w9','w10', 'w11',   
    'd0','d1','d2', 'd3','d4', 'd5','d6', 'd7','d8', 'd9',   
    'p0','p1','p2', 'p3','p4', 'p5','p6', 'p7','p8', 'p9','p10', 'p11', 'p12',    
    'c0','c1','c2', 'c3','c4', 'c5','c6', 'c7','c8', 'c9','c10', 'c11', 'c12', 'c13',    
    'g0','g1','g2', 'g3','g4',
    'week_of_year',    
    'imdb_user_rating',
    'revenue'
]]

In [19]:
df_4a.head(2)

,budget,w0,w1,w2,w3,w4,w5,w6,w7,w8,...,c12,c13,g0,g1,g2,g3,g4,week_of_year,imdb_user_rating,revenue
0,17000000,-0.112276,-0.076632,0.032137,0.041747,0.088295,0.130720,0.024177,-0.048131,0.041176,...,-0.017277,0.026549,-0.490822,-0.090693,-0.207875,-0.176953,0.181547,40,6,30331165
1,0,-0.116457,-0.055080,0.038783,0.046381,0.085347,0.133306,0.023528,-0.046033,0.039759,...,-0.011548,0.025018,-0.411669,0.173315,-0.066780,-0.122879,0.591124,2,6,0


In [20]:
df_4a.to_csv(dir_1 + 'movie_processed_1.csv', index=False)